In [1]:
import pandas as pd
import os

#### in the goden dataset, ideally should have the same row number (no. of users) as the full_data set
#### even if there's no legitimate first name of that user, in the goden dataset, that user (row) should be kept, with no gender prediction 

In [2]:
%%time
df = pd.read_excel('./clean_Json_updated_debug_Dec_17_2017.xlsx')
print df.shape
gd = pd.read_csv('./users.gender.golden (3).csv')
print gd.shape

(195185, 40)
(166942, 8)
CPU times: user 1min 45s, sys: 936 ms, total: 1min 46s
Wall time: 1min 46s


<string>:2: DtypeWarning: Columns (1,7) have mixed types. Specify dtype option on import or set low_memory=False.


In [35]:
df['public_firstname'].isnull().sum()

0

In [34]:
df['public_firstname'].value_counts().head()

Muhammad    1026
Md           541
Amit         312
Daniel       309
John         300
Name: public_firstname, dtype: int64

#### full data have 195185 unique users

In [5]:
len(df['username'].unique()), len(df['userId'].unique())

(195185, 195185)

In [10]:
df.head(1).transpose()

,0
index,rrpteltd.html
about.avatar,/img/unknown.png
about.avatar_cdn,//cdn6.f-cdn.com/img/unknown.png
about.avatar_large,/img/unknown.png
about.avatar_large_cdn,//cdn6.f-cdn.com/img/unknown.png
about.chosen_role,2
about.company,None
about.cover_image,{u'current_image': {u'url': u'//cdn2.f-cdn.com...
about.display_name,RRPteLtd
about.hourly_rate,None


In [7]:
gd.head()

,user_id,username,freq,male.prob,fname.gender,male.prob2,fname.gender2,pic.gender
0,5,billion,2.0,1.0000,male,0.920000,male,NaN
1,63,paddyshyam,NaN,NaN,NaN,NaN,NaN,NaN
2,64,ArmenSoft,1.0,1.0000,male,NaN,NaN,NaN
3,128,okbookman,1.0,0.9945,male,0.998834,male,NaN
4,215,revitupmedia,1.0,1.0000,male,NaN,NaN,male


#### golden set has 166942 unique users

In [8]:
len(gd['username'].unique()), len(gd['user_id'].unique())

(119819, 166942)

In [20]:
gd['user_id'].isnull().sum()

0

In [12]:
gd['username'].isnull().sum()

47124

#### there's mutual data 154574 between full_info and golden_set 

In [19]:
joined = df.set_index('userId').join(gd.set_index('user_id'), how='inner', lsuffix='data_')
joined.shape

(154574, 46)

In [22]:
right_j = df.set_index('userId').join(gd.set_index('user_id'), how='right', lsuffix='data_')
right_j.shape

(166942, 46)

#### 12368 users in goden set but not in full_set (why?)

In [28]:
right_j[right_j['index'].isnull()].shape

(12368, 46)

In [27]:
right_j[right_j['index'].isnull()].tail()

,index,about.avatar,about.avatar_cdn,about.avatar_large,about.avatar_large_cdn,about.chosen_role,about.company,about.cover_image,about.display_name,about.hourly_rate,...,timeOffset,usernamedata_,public_firstname,username,freq,male.prob,fname.gender,male.prob2,fname.gender2,pic.gender
user_id,,,,,,,,,,,,,,,,,,,,,
1809867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,matthewlarner,2.0,0.9997,male,0.999269,male,NaN
1810291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,kamano,0.0,NaN,NaN,NaN,NaN,NaN
1810765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,IndyBobby,1.0,0.9726,male,0.999187,male,NaN
1811317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,dimpol86,1.0,1.0000,male,1.000000,male,NaN
1811738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Alexa77,1.0,0.0000,female,0.000000,female,NaN


#### find why

In [30]:
df[df['userId']==1809867]

,index,about.avatar,about.avatar_cdn,about.avatar_large,about.avatar_large_cdn,about.chosen_role,about.company,about.cover_image,about.display_name,about.hourly_rate,...,resume.education,resume.exams,resume.experience,resume.publications,resume.show,skills,timeOffset,userId,username,public_firstname


https://www.freelancer.com/users/1809867.html

#### you can see the user is closed, actually in the full_set, I have removed most close users

### overall, I think the goden set was not generated from the lastest full_data set